# Lab4: Visualizing trends from Order History

### Google Data Studio

Google Data Studio is a cloud powered service that can be used to deliver insights into your data by combining many different data sources to quickly visualize your data. Visualizations that are created can be made into dashboards that the team can collaborate on. 

### Retreiving the Cloud SQL MySQL Database Host 

In this lab we will be importing sample inventory data, and order history from a CSV into a MySQL Database in Google Cloud SQL. To begin, make sure you are signed into your Google account and can access the [Google Cloud Console](https://console.cloud.google.com/).

From the search bar at the top of the page, type in `SQL`. Click the **SQL** result to open the Cloud SQL console view.

Observe the **inventory-db** MySQL instance that has been created for you by the Cloud Academy lab environment:

![observe MySQL instance](assets/lab4/20250422144021.png)

*Note*: You must wait until the green checkmark appears next to the database. It can take 20 minutes from when you started the lab for the instance to reach the ready status.

Copy the **Public IP address** and use it to overwrite the default `host` IP address in the code block in the following section.

#### Populating the Mock Data
Use the Python code block below to populate our mock data into the Cloud SQL Database.

Replace the **REPLACE_ME** placeholder below with the **Public IP address** of your Cloud SQL instance. 

In [ ]:
! pip install mysql-connector-python

import mysql.connector, csv
## Make sure the Host matches your Cloud SQL instance
host="REPLACE_ME"
mydb = mysql.connector.connect(
  host=host, 
  user="root",
  passwd="demotest123"
)

cursor = mydb.cursor()
cursor.execute("CREATE DATABASE IF NOT EXISTS inventory_datastudio")
mydb.database = "inventory_datastudio"
cursor.execute("CREATE TABLE IF NOT EXISTS product_table (ProductName varchar(50),Description varchar(255),ProductSku bigint(12),QuantityInStock int(5),ReorderLevel int(5),WholeSaleCost decimal(8,2),RetailCost decimal(8,2));")
cursor.execute("CREATE TABLE IF NOT EXISTS order_history (CustomerName varchar(50),OrderDate varchar(255),OrderId bigint(12));")
cursor.execute("CREATE TABLE IF NOT EXISTS orders (OrderId bigint(12),ProductSku bigint(12), Price decimal(8,2));")
cursor.execute("CREATE VIEW orders_and_order_history AS SELECT t1.CustomerName, t1.OrderDate, t2.OrderId, t2.ProductSku, t2.Price FROM order_history t1, orders t2 WHERE t1.OrderId = t2.OrderId")

print("Database setup complete.")
with open('./data_files/Lab4_Inventory.csv') as datafile:
    csv_data = csv.reader(datafile)
    next(csv_data) 
    for row in csv_data:
        cursor.execute ("INSERT INTO product_table (ProductName,Description,ProductSku,QuantityInStock,ReorderLevel,WholeSaleCost,RetailCost) VALUES (%s, %s, %s, %s, %s, %s, %s)",row)
    mydb.commit()
print("Success! Inventory Data inserted")

with open('./data_files/OrderHistory.csv') as datafile:
    csv_data = csv.reader(datafile)
    next(csv_data) 
    for row in csv_data:
        cursor.execute ("INSERT INTO order_history (CustomerName, OrderDate, OrderId) VALUES (%s, %s, %s)",row)
    mydb.commit()
print("Success! Order History Data inserted")

with open('./data_files/Orders.csv') as datafile:
    csv_data = csv.reader(datafile)
    next(csv_data) 
    for row in csv_data:
        cursor.execute ("INSERT INTO orders (OrderId, ProductSku, Price) VALUES (%s, %s, %s)",row)
    mydb.commit()
print("Success! Orders Data inserted")


mydb.close() 

#### **Lab:** Getting Started in GCP Data Studio
1. First [Navigate to Data Studio](https://datastudio.google.com/u/1/navigation/reporting).

2. Next click `Blank Report`.

![BlankReport](./assets/lab4/lab4_blank_report.png)

3. If prompted, configure the following in the account setup popup, clicking **continue** to proceed:

    - **Country**: Enter your country
    - **Company**: Enter *student*
    - Select **No** to the following updates:
        - **Tips and recommendations**: No
        - **Product announcements**: No
        - **Market research**: No

    ![](assets/20250422144434.png)
    
    ![](assets/20250422144534.png)

    Repeat the previous lab step if you are redirected to the Data Studio home page.

4. Navigate and find `MySql` in the Connect to data form that appears.

5. If prompted, click Authorize and select your lab user in the pop-up authorization form to allow Data Studio to connect to the data.

6. Enter the following in the `BASIC` tab to connect to the database:
    
    - **Host Name or IP**: Enter the public IP address from Cloud SQL
    - **Database**: _inventory_datastudio_
    - **Username**: _root_
    - **Password**: _demotest123_

7. Click `Authenticate` and select the `orders_and_order_history` view.  
![Authenticate](assets/20250422145621.png)

8. Click `Add` and, if prompted, `Add to report` in the pop-up to add the data into the report.

9. Next we will create a quick report on which customer's had the most expensive orders. On the rightmost bar you can select both the dimension and metric.

10. Drag the **CustomerName** field from the **Data** sidebar onto the canvas. This will create a chart named CustomerName:

![](assets/20250422150154.png)

11. With the chart still selected, click **Add metric** in the **Chart** panel and select **Price**:

![](assets/20250422150258.png)

![](assets/20250422150312.png)

The chart will update with each customers total order price.

12. Take a look at the results, we can see John Smith had the largest order.  

Feel free to adjust the other dimensions to understand some insights of the order table.

#### **Lab:** Finding the Most Commonly Sold Product

In this lab we will add a separate table, perform data blending to join from the inventory table to the `orders_and_order_history` view, and find out which product we sell the most.

1) Select `Add Data` to add another table to Data Studio.

![Add Data](./assets/lab4/lab4_add_data.png)
![](assets/20250422150454.png)

2) Add a `MySql` data source.  Enter the same database configuration as before, then click **Authenticate**.

3) This time select `product_table`, followed by **Add**.

4) Click **Manage blends** from the **Resource** menu, and click **Add a blend**:

![](assets/20250422150946.png)

![](assets/20250422150955.png)

5) Select `Blend Data` to open the blend data wizard.

![Blend Data](./assets/lab4/lab4_blend.png)

6) Click `Join another table` and add the recently added table from the previous steps.  Ensure the join keys have `ProductSku` and that the inventory table has `ProductName` included.  Ensure in the `orders_and_order_history` datastore has count selected for `ProductSku`. Click `Configure join` and ensure `Left outer` is selected and `ProductSku Table 1` and `ProductSku Table 2` are listed as `Join conditions`.

![datasource join](./assets/lab4/lab4_join_config.png)

![](assets/20250422151411.png)

7) Click `Save`

8) Create a new chart with the dimensions, `ProductName` and `ProductSku`, a metric of sum for `ProductSku` , and a sort by `ProductSku` _descending_.

![](assets/20250422151900.png)

We have successfully created a chart that shows are most sold products!  Your graph should look similar to below.

![](assets/20250422151924.png)

#### **Lab:** Other Insights on Data

1) Click **Manage blends** from the **Resource** menu, and click **Remove**, followed by **Remove blend** to delete the blended data source:

![](assets/20250422152229.png)

![Add Data](./assets/lab4/lab4_remove_blended.png)

2) Click `Select data source` and choose the 1st MySQL data source in the list representing the products table you added.

![Select data source](./assets/lab4/lab4_prior_data_source.png)

3) Attempt to create the following chart to find the most expensive product that is sold:

![Most expensive product](./assets/lab4/lab4_most_expensive.png)

Review the following setup to ensure your chart is correct:

![](assets/20250422152720.png)

4) Next we will find the total retail value of our inventory.  Select `Add a chart` above the editor:

![add chart](./assets/lab4/lab4_add_chart.png)

5) Select the `Total` type `Scorecard`:

![](assets/20250422152828.png)

6) The scorecard chart will default to the `RetailCost` metric.

![sum chart](./assets/lab4/lab4_chart_sum.png)

### Cleaning Up

For the sake of this lab, when you are finished exploring the data in data studio, you will clean up the Google Data Studio envrionment. That is because the Cloud SQL database is deleted once the lab session ends, making the reports and data sources unusable in Data Studio.

1. Click the Data Studio icon in the upper-left corner to return to the Data Studio home view.

![](assets/20250422153002.png)

2. Click the three dots to the right of your report followed by `Remove`.

![Remove report](./assets/lab4/lab4_remove_report.png)

3. Similarly, remove the two data sources on the `Data sources` tab.

4. The report and data sources are moved to the `Trash` as a result. To permanantly delete them, open the trash and click `Delete forever` for all of the items.